In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision.models import efficientnet_b0
from sklearn.metrics import classification_report
import numpy as np
from google.colab import drive

In [2]:
# Config device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
# Monter Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Vérifier le contenu du dossier
data_path = "/content/drive/MyDrive/image/image"
print("Dossiers/classes dans data_path :", os.listdir(data_path))

Dossiers/classes dans data_path : ['yorkshire_terrier', 'Siamese', 'samoyed', 'scottish_terrier', 'Russian_Blue', 'shiba_inu', 'staffordshire_bull_terrier', 'wheaten_terrier', 'Sphynx', 'saint_bernard', 'pug', 'Maine_Coon', 'Persian', 'pomeranian', 'japanese_chin', 'miniature_pinscher', 'newfoundland', 'keeshond', 'Ragdoll', 'leonberger', 'great_pyrenees', 'Bombay', 'german_shorthaired', 'havanese', 'chihuahua', 'boxer', 'British_Shorthair', 'english_setter', 'english_cocker_spaniel', 'Egyptian_Mau', 'american_bulldog', 'beagle', 'Bengal', 'basset_hound', 'Birman', 'Abyssinian', 'american_pit_bull_terrier']


In [5]:
# Transformation des images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [6]:
# Chargement du dataset
dataset = ImageFolder(data_path, transform=transform)
print("Classes détectées :", dataset.class_to_idx)
print("Nombre total d'images :", len(dataset))

Classes détectées : {'Abyssinian': 0, 'Bengal': 1, 'Birman': 2, 'Bombay': 3, 'British_Shorthair': 4, 'Egyptian_Mau': 5, 'Maine_Coon': 6, 'Persian': 7, 'Ragdoll': 8, 'Russian_Blue': 9, 'Siamese': 10, 'Sphynx': 11, 'american_bulldog': 12, 'american_pit_bull_terrier': 13, 'basset_hound': 14, 'beagle': 15, 'boxer': 16, 'chihuahua': 17, 'english_cocker_spaniel': 18, 'english_setter': 19, 'german_shorthaired': 20, 'great_pyrenees': 21, 'havanese': 22, 'japanese_chin': 23, 'keeshond': 24, 'leonberger': 25, 'miniature_pinscher': 26, 'newfoundland': 27, 'pomeranian': 28, 'pug': 29, 'saint_bernard': 30, 'samoyed': 31, 'scottish_terrier': 32, 'shiba_inu': 33, 'staffordshire_bull_terrier': 34, 'wheaten_terrier': 35, 'yorkshire_terrier': 36}
Nombre total d'images : 10573


In [7]:
# Split train / validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [8]:
# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [9]:

# Charger EfficientNet pré-entraîné
model = efficientnet_b0(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 179MB/s]


In [10]:
# Adapter le classifier au nombre de classes
nb_classes = len(dataset.classes)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, nb_classes)
model = model.to(device)

In [11]:
# Définir loss et optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Scheduler pour réduire le LR quand ça stagne
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:


# Entraînement
num_epochs = 30
patience = 5
best_val_loss = np.inf
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total = 0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss = val_loss / val_total
    val_acc = val_correct / val_total

    scheduler.step(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# Afficher un batch d'exemple
for images, labels in train_loader:
    print("Exemple de labels dans un batch :", labels)
    break


Using device: cuda
Mounted at /content/drive
Dossiers/classes dans data_path : ['yorkshire_terrier', 'Siamese', 'samoyed', 'scottish_terrier', 'Russian_Blue', 'shiba_inu', 'staffordshire_bull_terrier', 'wheaten_terrier', 'Sphynx', 'saint_bernard', 'pug', 'Maine_Coon', 'Persian', 'pomeranian', 'japanese_chin', 'miniature_pinscher', 'newfoundland', 'keeshond', 'Ragdoll', 'leonberger', 'great_pyrenees', 'Bombay', 'german_shorthaired', 'havanese', 'chihuahua', 'boxer', 'British_Shorthair', 'english_setter', 'english_cocker_spaniel', 'Egyptian_Mau', 'american_bulldog', 'beagle', 'Bengal', 'basset_hound', 'Birman', 'Abyssinian', 'american_pit_bull_terrier']


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


Classes détectées : {'Abyssinian': 0, 'Bengal': 1, 'Birman': 2, 'Bombay': 3, 'British_Shorthair': 4, 'Egyptian_Mau': 5, 'Maine_Coon': 6, 'Persian': 7, 'Ragdoll': 8, 'Russian_Blue': 9, 'Siamese': 10, 'Sphynx': 11, 'american_bulldog': 12, 'american_pit_bull_terrier': 13, 'basset_hound': 14, 'beagle': 15, 'boxer': 16, 'chihuahua': 17, 'english_cocker_spaniel': 18, 'english_setter': 19, 'german_shorthaired': 20, 'great_pyrenees': 21, 'havanese': 22, 'japanese_chin': 23, 'keeshond': 24, 'leonberger': 25, 'miniature_pinscher': 26, 'newfoundland': 27, 'pomeranian': 28, 'pug': 29, 'saint_bernard': 30, 'samoyed': 31, 'scottish_terrier': 32, 'shiba_inu': 33, 'staffordshire_bull_terrier': 34, 'wheaten_terrier': 35, 'yorkshire_terrier': 36}
Nombre total d'images : 10573


100%|██████████| 20.5M/20.5M [00:00<00:00, 149MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/30], Train Loss: 0.9463, Train Acc: 0.7229, Val Loss: 0.6230, Val Acc: 0.8123
Epoch [2/30], Train Loss: 0.4613, Train Acc: 0.8522, Val Loss: 0.5231, Val Acc: 0.8444
Epoch [3/30], Train Loss: 0.2955, Train Acc: 0.9075, Val Loss: 0.4659, Val Acc: 0.8667
Epoch [4/30], Train Loss: 0.2365, Train Acc: 0.9260, Val Loss: 0.5680, Val Acc: 0.8454
Epoch [5/30], Train Loss: 0.2052, Train Acc: 0.9346, Val Loss: 0.5967, Val Acc: 0.8459
Epoch [6/30], Train Loss: 0.1698, Train Acc: 0.9454, Val Loss: 0.5552, Val Acc: 0.8638
Epoch [7/30], Train Loss: 0.0592, Train Acc: 0.9817, Val Loss: 0.4564, Val Acc: 0.8846
Epoch [8/30], Train Loss: 0.0231, Train Acc: 0.9926, Val Loss: 0.4271, Val Acc: 0.8931
Epoch [9/30], Train Loss: 0.0161, Train Acc: 0.9957, Val Loss: 0.4922, Val Acc: 0.8931
Epoch [10/30], Train Loss: 0.0184, Train Acc: 0.9944, Val Loss: 0.4509, Val Acc: 0.8955
Epoch [11/30], Train Loss: 0.0224, Train Acc: 0.9933, Val Loss: 0.5116, Val Acc: 0.8931
Epoch [12/30], Train Loss: 0.0128, Train 